In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../data/raw/CVD_Cleaned.csv')

In [4]:
df_cleaned = df.copy()

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import SGDClassifier

In [6]:
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [7]:
cat_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse_output=False, drop='first'))
    ]
)

In [8]:
age_ord_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder())
    ]
)

In [9]:
health_ord_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder(categories=[['Poor','Fair','Good','Very Good','Excellent']]))
    ]
)

In [10]:
checkup_ord_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder(categories=[['Within the past year','Within the past 2 years','Within the past 5 years','5 or more years ago','Never']]))
    ]
)

In [11]:
num_cols = df_cleaned.select_dtypes(include=['float64']).columns

In [12]:
cat_cols = ['Arthritis', 'Depression', 'Diabetes', 'Exercise', 'Other_Cancer', 'Sex', 'Skin_Cancer', 'Smoking_History']

In [13]:
from sklearn.compose import ColumnTransformer

In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat_cols', cat_transformer, cat_cols),
        ('age_col', age_ord_transformer, ['Age_Category']),
        ('health_col', health_ord_transformer, ['General_Health']),
        ('checkup_col', checkup_ord_transformer, ['Checkup'])
    ]
)

In [15]:
sgd_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('sgd', SGDClassifier(loss='log_loss', penalty='elasticnet', random_state=42))
    ]
)

In [16]:
target = df_cleaned['Heart_Disease'].map({'Yes': 1, 'No': 0})
sgd_pipe.fit(df_cleaned, target)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption',
       'Fruit_Consumption', 'Green_Vegetables_Consumption',
       'FriedPotato_Consumption'],
      dtype='object')),
                                                 ('cat_cols',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='fir...
                                                                                               'Good',
                                                                                               'Very '
                                                                                               'Good',
                                                                                               'Excellent']]))]),
                                                  ['General_Health']),
                                                 ('checkup_col',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder(categories=[['Within '
                                                                                               'the '
                                                                                               'past '
                                                                                               'year',
                                                                                               'Within '
                                                                                               'the '
                                                                                               'past '
                                                                                               '2 '
                                                                                               'years',
                                                                                               'Within '
                                                                                               'the '
                                                                                               'past '
                                                                                               '5 '
                                                                                               'years',
                                                                                               '5 '
                                                                                               'or '
                                                                                               'more '
                                                                                               'years '
                                                                                               'ago',
                                                                                               'Never']]))]),
                                                  ['Checkup'])])),
                ('sgd',
                 SGDClassifier(loss='log_loss', penalty='elasticnet',
                               random_state=42))])

In [17]:
sgd_pipe.predict(df_cleaned)

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
obs = pd.DataFrame(df_cleaned.iloc[0]).transpose()
sgd_pipe.predict(obs)

array([0])

In [19]:
from joblib import dump

dump(sgd_pipe,  '../models/sgd_pipeline.joblib')

['../models/sgd_pipeline.joblib']